In [ ]:
import tensorflow.compat.v1 as tf #2차원
tf.debugging.set_log_device_placement(True)
tf.disable_v2_behavior()
tf.reset_default_graph()
import numpy as np
import random as rd
#from google.colab import drive
#drive.mount('/content/gdrive')

K=int(4)
deltat=0.5 #1
T=20 #100
delta=deltat*(K+1)
N=int(T/deltat)
pmax=40*deltat
pavg=10*deltat 
beta=1
zeta=1
noisea=10**(-8)
noise=10**(-8)
V=5
H=5
L=V*deltat

inputs=9
output=(3+K)*N #
node_num = output*2  
hidden_nodesL2=node_num     
hidden_nodesL3=node_num
hidden_nodesL4=node_num    
hidden_nodesL5=node_num    
hidden_nodesL6=node_num    


X=tf.placeholder(tf.float32,[None,inputs])
# hidden layer의 node수를 output의 2배 이상
W1=tf.get_variable("W1",shape=[inputs,hidden_nodesL2],initializer=tf.keras.initializers.he_normal())
b1=tf.get_variable("b1",shape=[hidden_nodesL2],initializer=tf.keras.initializers.he_normal())
L1=tf.nn.relu(tf.matmul(X,W1)+b1)
L1 = tf.nn.experimental.stateless_dropout(L1, rate=0.1, seed=[1, 0])

W2=tf.get_variable("W2",shape=[hidden_nodesL2,hidden_nodesL3],initializer=tf.keras.initializers.he_normal())
b2=tf.get_variable("b2",shape=[hidden_nodesL3],initializer=tf.keras.initializers.he_normal())
L2=tf.nn.relu(tf.matmul(L1,W2)+b2)
L2 = tf.nn.experimental.stateless_dropout(L2, rate=0.1, seed=[1, 0])

W3=tf.get_variable("W3",shape=[hidden_nodesL3,hidden_nodesL4],initializer=tf.keras.initializers.he_normal())
b3=tf.get_variable("b3",shape=[hidden_nodesL4],initializer=tf.keras.initializers.he_normal())
L3=tf.nn.relu(tf.matmul(L2,W3)+b3)
L3 = tf.nn.experimental.stateless_dropout(L3, rate=0.1, seed=[1, 0])

W4=tf.get_variable("W4",shape=[hidden_nodesL4,hidden_nodesL5],initializer=tf.keras.initializers.he_normal())
b4=tf.get_variable("b4",shape=[hidden_nodesL5],initializer=tf.keras.initializers.he_normal())
L4=tf.nn.relu(tf.matmul(L3,W4)+b4)
L4 = tf.nn.experimental.stateless_dropout(L4, rate=0.1, seed=[1, 0])

W5=tf.get_variable("W5",shape=[hidden_nodesL5,hidden_nodesL6],initializer=tf.keras.initializers.he_normal())
b5=tf.get_variable("b5",shape=[hidden_nodesL6],initializer=tf.keras.initializers.he_normal())
L5=tf.nn.relu(tf.matmul(L4,W5)+b5)
L5 = tf.nn.experimental.stateless_dropout(L5, rate=0.1, seed=[1, 0])

W6=tf.get_variable("W6",shape=[hidden_nodesL6,output],initializer=tf.keras.initializers.he_normal())
b6=tf.get_variable("b6",shape=[output],initializer=tf.keras.initializers.he_normal())
Loutput=tf.nn.sigmoid(tf.matmul(L5,W6)+b6)
# batch X (K+3)N

# Loutput - > 1batch X (K+3)N
bch_size=tf.shape(Loutput)[0]

tmp_P=Loutput[:,:N]#N개
tmp_a=Loutput[:,N:(K+1)*N]#KN개
tmp_qx=Loutput[:,(K+1)*N:(K+2)*N]#N개
tmp_qy=Loutput[:,(K+2)*N:(K+3)*N]#N개

P=tf.reshape(pmax*tmp_P,[bch_size,N]) #정규화된 output을 다시 원래 크기로
alpha=tf.reshape(tmp_a,[bch_size,K*N])
qx=tf.reshape(20*tmp_qx,[bch_size,N])
qy=tf.reshape(10*tmp_qy,[bch_size,N])

nodex=tf.reshape(X[:,:4]*20,[bch_size,4]) # input
nodey=tf.reshape(X[:,4:8]*10,[bch_size,4])
Rmin=tf.reshape(X[:,8:9]*15,[bch_size,1]) 

nodex1 = tf.reshape(nodex[:,0],[bch_size,1]) #user node W를 늘리기
nodex11 = tf.reshape(nodex[:,0],[bch_size,1])
nodex2 = tf.reshape(nodex[:,1],[bch_size,1])
nodex22 = tf.reshape(nodex[:,1],[bch_size,1])
nodex3 = tf.reshape(nodex[:,2],[bch_size,1])
nodex33 = tf.reshape(nodex[:,2],[bch_size,1])
nodex4 = tf.reshape(nodex[:,3],[bch_size,1])
nodex44 = tf.reshape(nodex[:,3],[bch_size,1])

nodey1 = tf.reshape(nodey[:,0],[bch_size,1])
nodey11 = tf.reshape(nodey[:,0],[bch_size,1])
nodey2 = tf.reshape(nodey[:,1],[bch_size,1])
nodey22 = tf.reshape(nodey[:,1],[bch_size,1])
nodey3 = tf.reshape(nodey[:,2],[bch_size,1])
nodey33 = tf.reshape(nodey[:,2],[bch_size,1])
nodey4 = tf.reshape(nodey[:,3],[bch_size,1])
nodey44 = tf.reshape(nodey[:,3],[bch_size,1])

for j in range(N-1):
  nodex11 = tf.concat([nodex11,nodex1],axis=1) # batch x N
  nodex22 = tf.concat([nodex22,nodex2],axis=1)
  nodex33 = tf.concat([nodex33,nodex3],axis=1)
  nodex44 = tf.concat([nodex44,nodex4],axis=1)
  nodey11 = tf.concat([nodey11,nodey1],axis=1)
  nodey22 = tf.concat([nodey22,nodey2],axis=1)
  nodey33 = tf.concat([nodey33,nodey3],axis=1)
  nodey44 = tf.concat([nodey44,nodey4],axis=1)

h1=beta/(tf.add(tf.pow(tf.subtract(qx,nodex11),2),tf.pow(tf.subtract(qy,nodey11),2))+H**2) # batch x N
h2=beta/(tf.add(tf.pow(tf.subtract(qx,nodex22),2),tf.pow(tf.subtract(qy,nodey22),2))+H**2)
h3=beta/(tf.add(tf.pow(tf.subtract(qx,nodex33),2),tf.pow(tf.subtract(qy,nodey33),2))+H**2)
h4=beta/(tf.add(tf.pow(tf.subtract(qx,nodex44),2),tf.pow(tf.subtract(qy,nodey44),2))+H**2)

#H=tf.concat([h1,h2,h3,h4],0) 

a=tf.zeros([bch_size,1])
qxx=tf.concat([a,qx],1) # |q[N+1]-q[0]| =0     batch x (N+1)
qyy=tf.concat([a,qy],1)

#constraint (11) -> 모든 N을 더한 downlink rate가 모든 k에 대해서 만족해야 된다.
RD1=tf.log(1+tf.divide(tf.multiply(tf.multiply((1-alpha[:,:N]),h1),P),((1-alpha[:,:N])*noisea+noise)))/np.log(2) # 100 X 1 X 20
RD2=tf.log(1+tf.divide(tf.multiply(tf.multiply((1-alpha[:,N:2*N]),h2),P),((1-alpha[:,N:2*N])*noisea+noise)))/np.log(2)
RD3=tf.log(1+tf.divide(tf.multiply(tf.multiply((1-alpha[:,2*N:3*N]),h3),P),((1-alpha[:,2*N:3*N])*noisea+noise)))/np.log(2)
RD4=tf.log(1+tf.divide(tf.multiply(tf.multiply((1-alpha[:,3*N:4*N]),h4),P),((1-alpha[:,3*N:4*N])*noisea+noise)))/np.log(2)
RD1m=tf.reduce_mean(tf.subtract(Rmin,tf.math.reduce_mean(RD1,axis = 1 , keep_dims=True)),0) # 100 X 1 로 일단 계산
RD2m=tf.reduce_mean(tf.subtract(Rmin,tf.reduce_mean(RD2,axis = 1 , keep_dims=True)),0) 
RD3m=tf.reduce_mean(tf.subtract(Rmin,tf.reduce_mean(RD3,axis = 1 , keep_dims=True)),0) 
RD4m=tf.reduce_mean(tf.subtract(Rmin,tf.reduce_mean(RD4,axis = 1 , keep_dims=True)),0) 
RD1_real = tf.maximum(RD1m,0) # batch를 평균 내기  scalar
RD2_real = tf.maximum(RD2m,0)
RD3_real = tf.maximum(RD3m,0)
RD4_real = tf.maximum(RD4m,0) 

#min RU 고르기
E1=tf.multiply(tf.multiply(h1,alpha[:,:N]),P)*deltat # batch x N
E2=tf.multiply(tf.multiply(h2,alpha[:,N:2*N]),P)*deltat
E3=tf.multiply(tf.multiply(h3,alpha[:,2*N:3*N]),P)*deltat
E4=tf.multiply(tf.multiply(h4,alpha[:,3*N:4*N]),P)*deltat
RU1=tf.reduce_mean(tf.log(1+tf.multiply(h1,E1)/((noisea+noise)*deltat))/np.log(2)) 
RU2=tf.reduce_mean(tf.log(1+tf.multiply(h2,E2)/((noisea+noise)*deltat))/np.log(2))
RU3=tf.reduce_mean(tf.log(1+tf.multiply(h3,E3)/((noisea+noise)*deltat))/np.log(2))
RU4=tf.reduce_mean(tf.log(1+tf.multiply(h4,E4)/((noisea+noise)*deltat))/np.log(2))

a_1=tf.minimum(RU1,RU2)
b=tf.minimum(a_1,RU3)
c=tf.minimum(b,RU4)
RUmin = c

qx_init = tf.reduce_mean(qx[:,0:1])
qx_finish = tf.reduce_mean(qx[:,N-1:N])
qy_init = tf.reduce_mean(qy[:,0:1])
qy_finish = tf.reduce_mean(qy[:,N-1:N])
# constraint (2)
P_sub1=tf.reduce_mean(P,axis = 1 , keep_dims=True)-pavg
P_sub2=tf.reduce_mean(P-pmax)

C1= tf.reduce_mean(tf.maximum(tf.sqrt(tf.add(tf.pow(tf.subtract(qx[:,1:N],qxx[:,1:N]),2),\
                                             tf.pow(tf.subtract(qy[:,1:N],qyy[:,1:N]),2)))-L,0))
c1_con =tf.reduce_mean((tf.sqrt(tf.add(tf.pow(tf.subtract(qx[:,1:N],qxx[:,1:N]),2),\
                                             tf.pow(tf.subtract(qy[:,1:N],qyy[:,1:N]),2)))-L))
C2=tf.maximum(tf.reduce_mean(P_sub1),0) #constraint (4)
c2_con =tf.reduce_mean(P_sub1)
C3=tf.maximum(tf.reduce_mean(P_sub2),0) #constraint (5)
c3_con =tf.reduce_mean(P_sub2)
C4=(RD1_real+RD2_real+RD3_real+RD4_real) # 이미 max는 적용돼있다. constraint (11)
c4_con = RD1m+RD2m+RD3m+RD4m
C5=tf.maximum(tf.abs(tf.subtract(qx_init,qx_finish))+tf.abs(tf.subtract(qy_init,qy_finish)),0)  # constraint (3)
c5_con1 = (tf.abs(tf.subtract(qx_init,qx_finish))+tf.abs(tf.subtract(qy_init,qy_finish)))
#Reward=-RUmin+C1+C2+C3+C4+C5
Reward=-tf.exp(RUmin)+(C1*50+C2*10000+C3+C5+C4)*35000 #기하평균? 벌점을 줄이면 rate는 높아진다?
optimizer=tf.train.AdamOptimizer(learning_rate=0.001).minimize(Reward) #0.001 -> 0.01

batch_size=512 #512

sess = tf.Session()
sess.run(tf.global_variables_initializer())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
print(tf.config.list_physical_devices('GPU')) # TensorFlow가 GPU를 사용하고 있는지 확인
with tf.device('/cpu:0'):
  print(tf.config.list_physical_devices('GPU')) # TensorFlow가 GPU를 사용하고 있는지 확인
  for cnt in range(10000):#300 # 이거 안되면 초기 궤적 집어넣기,  (현재 GPU + batch size 100 + dropout + learning rate 0.01은 하면 안됨..)
    hh=[] 
    for j in range(batch_size): #batch 만큼 sample 생성
        x=[0 for i in range(9)] # sample 생성   
        for i in range(inputs):
            if i<4:
                x[i]=float(rd.randint(0,20))/20
            elif i<8:
                x[i]=float(rd.randint(0,10))/10
            elif i ==8:
                x[i]=15.0/15#float(rd.randint(5,15)) #/15
            '''elif i<=29 : # 9 ~ 28 
                x[i] = 10*np.cos(2*np.pi*(i-10)/N)+10
            else : # 29 ~ 48
                x[i] = 5*np.sin(2*np.pi*(i-30)/N)+5'''
        hh.append(x)
        if cnt == 5000 and j==2:
          Htest = x
    hh = np.array(hh)
    hh = np.reshape(hh,(batch_size, inputs))
    
    for epoch in range(1):#1500
      with tf.device('/device:GPU:0'):
        c1_1,c2_1,c3_1,c4_1,rate,reward,_=sess.run([c1_con,c2_con,c3_con,c4_con,RUmin,Reward,optimizer],feed_dict={X:hh})
        
    #f tf.mod(cnt,1000)==0:
    print("Loop",cnt,'Reward',reward,'rate',rate)
    print("c1",c1_1,"c2",c2_1,"c3",c3_1,"c4",c4_1)

In [ ]:
test_input = [0,10/20,20/20,10/20,5/10,0,5/10,10/10,15/15] #/20/10/15
'''for i in range(9,49):
    if i<=29 : # 10 ~ 29 
        test_input.append(10*np.cos(2*np.pi*(i-10)/N)+10)
    else : # 30 ~ 49
        test_input.append(5*np.sin(2*np.pi*(i-30)/N)+5)'''
test_input = np.reshape(test_input,(1, 9))
#Htest = np.reshape(Htest,(1,9))
with tf.device('/cpu:0'):
  for i in range(10000):
    alphaa,c1,c2,c3,c4,ru1,ru2,ru3,ru4,test_rate,test_reward,test_qx,test_qy,test_p,_\
    =sess.run([alpha,c1_con,c2_con,c3_con,c4_con,RU1,RU2,RU3,RU4,RUmin,Reward,qx,qy,P,optimizer],\
              feed_dict={X:test_input})#rate, 벌점을 준 rate, qx, qy, p



In [ ]:
print(test_p)
print(c1,c2,c3,c4)

In [ ]:
print(test_rate)
#print(list(test_p))
###print(list(alpha))
print(list(test_qx))
print(list(test_qy))
#print(len(test_qx))

In [ ]:
import matplotlib.pyplot as plt
test_qx = list(test_qx)[0]
test_qy = list(test_qy)[0]
plt.xlim(0,20)
plt.ylim(0,10)
plt.plot(test_qx,test_qy)
#plt.scatter(test_qx,test_qy)
plt.show()